# About

This notebook will introduce the `DELETE` and `REMOVE` Cypher keywords

In [ ]:
from neo4j import GraphDatabase, Record, ResultSummary, EagerResult
from neo4j.time import Date

import pandas as pd
pd.set_option('display.max_colwidth', 100)

import os 
import sys
from dotenv import load_dotenv 
load_dotenv()

# Add the utils directory to sys.path
sys.path.append(os.path.abspath("../utils"))

from Neo4jParser import Neo4jParser


NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

## `DELETE`/`DETACH`

* `DELETE` lets you remove data from your graph whether it's nodes, relationships or both.
* To remove relationships you use `DETACH`.
* To clear the entire graph run: `MATCH (n) DETACH DELETE n`

In [5]:
# Create some records in database "dev"
result = driver.execute_query("CREATE DATABASE dev IF NOT EXISTS;", database_="dev")
result = driver.execute_query("CREATE (),(),(),(),(),() RETURN 1", database_="dev")

data = Neo4jParser.parse(result, True, False)
data

Started streaming 1 records after 217 ms and completed after 228 ms.

Query executed against database: 'dev': CREATE (),(),(),(),(),() RETURN 1


{'1': [1]}

In [7]:
# How many records are in the database?
result = driver.execute_query("MATCH (n) RETURN COUNT(n);", database_="dev")

data = Neo4jParser.parse(result, True, False)
data

Started streaming 1 records after 33 ms and completed after 34 ms.

Query executed against database: 'dev': MATCH (n) RETURN COUNT(n);


{'COUNT(n)': [29]}

In [8]:
# Remove all relationships and nodes in the graph
result = driver.execute_query("MATCH (n) DETACH DELETE n;", database_="dev")

data = Neo4jParser.parse(result, True, False)
data

Started streaming 0 records after 155 ms and completed after 155 ms.

Query executed against database: 'dev': MATCH (n) DETACH DELETE n;


{}

In [9]:
# How many records are in the database now?
result = driver.execute_query("MATCH (n) RETURN COUNT(n);", database_="dev")

data = Neo4jParser.parse(result, True, False)
data

Started streaming 1 records after 2 ms and completed after 2 ms.

Query executed against database: 'dev': MATCH (n) RETURN COUNT(n);


{'COUNT(n)': [0]}

In [23]:
# Let's create some fake data
result = driver.execute_query(
    """ 
    CREATE (p1:Person {name: 'John Doe', age: 30, city: 'New York'}),
        (p2:Person {name: 'Jane Smith', age: 25, city: 'Los Angeles'}),
        (p3:Person {name: 'Alice Johnson', age: 35, city: 'Chicago'}),
        (p4:Person {name: 'Bob Brown', age: 40, city: 'Miami'}),
        (p5:Person {name: 'Charlie Davis', age: 28, city: 'Boston'});
    """,
    database_="dev"
)

In [24]:
# Let's remove the name from Bob Brown. He requests we keep his name confidential.
result = driver.execute_query(
    """ 
    MATCH (p:Person {name: "Bob Brown"})
    REMOVE p.name
    RETURN p
    """,
    database_="dev"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 1 records after 0 ms and completed after 0 ms.

Query executed against database: 'dev':  
    MATCH (p:Person {name: "Bob Brown"})
    REMOVE p.name
    RETURN p
    


{'p': [{'elementId': '4:ff73e06d-56ad-4959-b409-fcc3d9dce978:21',
   'labels': frozenset({'Person'}),
   'properties': {'city': 'Miami', 'age': 40}}]}

In [25]:
# Remove a label from a node
result = driver.execute_query(
    """ 
    MATCH (p:Person)
    REMOVE p:Person
    RETURN *
    """,
    database_="dev"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 5 records after 0 ms and completed after 0 ms.

Query executed against database: 'dev':  
    MATCH (p:Person)
    REMOVE p:Person
    RETURN *
    


{'p': [{'elementId': '4:ff73e06d-56ad-4959-b409-fcc3d9dce978:18',
   'labels': frozenset(),
   'properties': {'city': 'New York', 'name': 'John Doe', 'age': 30}},
  {'elementId': '4:ff73e06d-56ad-4959-b409-fcc3d9dce978:19',
   'labels': frozenset(),
   'properties': {'city': 'Los Angeles', 'name': 'Jane Smith', 'age': 25}},
  {'elementId': '4:ff73e06d-56ad-4959-b409-fcc3d9dce978:20',
   'labels': frozenset(),
   'properties': {'city': 'Chicago', 'name': 'Alice Johnson', 'age': 35}},
  {'elementId': '4:ff73e06d-56ad-4959-b409-fcc3d9dce978:21',
   'labels': frozenset(),
   'properties': {'city': 'Miami', 'age': 40}},
  {'elementId': '4:ff73e06d-56ad-4959-b409-fcc3d9dce978:22',
   'labels': frozenset(),
   'properties': {'city': 'Boston', 'name': 'Charlie Davis', 'age': 28}}]}

In [26]:
# Remove the age property 
result = driver.execute_query(
    """ 
    MATCH (p)
    REMOVE p.age
    RETURN *
    """,
    database_="dev"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 15 records after 33 ms and completed after 34 ms.

Query executed against database: 'dev':  
    MATCH (p)
    REMOVE p.age
    RETURN *
    


{'p': [{'elementId': '4:ff73e06d-56ad-4959-b409-fcc3d9dce978:8',
   'labels': frozenset(),
   'properties': {'city': 'New York', 'name': 'John Doe'}},
  {'elementId': '4:ff73e06d-56ad-4959-b409-fcc3d9dce978:9',
   'labels': frozenset(),
   'properties': {'city': 'Los Angeles', 'name': 'Jane Smith'}},
  {'elementId': '4:ff73e06d-56ad-4959-b409-fcc3d9dce978:10',
   'labels': frozenset(),
   'properties': {'city': 'Chicago', 'name': 'Alice Johnson'}},
  {'elementId': '4:ff73e06d-56ad-4959-b409-fcc3d9dce978:11',
   'labels': frozenset(),
   'properties': {'city': 'Miami'}},
  {'elementId': '4:ff73e06d-56ad-4959-b409-fcc3d9dce978:12',
   'labels': frozenset(),
   'properties': {'city': 'Boston', 'name': 'Charlie Davis'}},
  {'elementId': '4:ff73e06d-56ad-4959-b409-fcc3d9dce978:13',
   'labels': frozenset(),
   'properties': {'city': 'New York', 'name': 'John Doe'}},
  {'elementId': '4:ff73e06d-56ad-4959-b409-fcc3d9dce978:14',
   'labels': frozenset(),
   'properties': {'city': 'Los Angeles'